原文代码作者：https://github.com/wzyonggege/statistical-learning-method

# 第4章 朴素贝叶斯

基于贝叶斯定理与特征条件独立假设的分类方法。

模型：

- 高斯模型
- 多项式模型
- 伯努利模型

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [2]:
# 创建数据集
def create_data():
    # 加载鸢尾花数据集
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, :])
    return data[:,:-1], data[:,-1]

In [3]:
# 交叉验证
X, y = create_data()
# 对数据集进行切分，生成70%的训练数据和30%的测试数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
# 查看测试集中第一条数据
X_test[0], y_test[0]

(array([5.4, 3. , 4.5, 1.5]), 1.0)

参考：https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

## GaussianNB 高斯朴素贝叶斯

特征的可能性被假设为高斯

概率密度函数：
$$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$$

数学期望(mean)：$\mu$，方差：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$

In [5]:
class NaiveBayes:
    def __init__(self):
        self.model = None
    
    # 计算期望（即平均值）
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))

    # 计算标准差（即方差）
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x-avg, 2) for x in X]) / float(len(X)))
    
    # 计算概率密度（概率密度函数）
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
    
    # 处理X_train训练数据集，计算每个特征值的期望和标准差
    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        return summaries

    # 分类别求出期望和标准差
    def fit(self, X, y):
        # 得到类别（即标签）
        labels = list(set(y))
        data = {label:[] for label in labels}
        # 将数据进行分类别存放
        for f, label in zip(X, y):
            data[label].append(f)
        # 计算每个分类下每个特征值的期望和标准差
        self.model = {label: self.summarize(value) for label, value in data.items()}
        return 'gaussianNB train done!'
    
    # 计算概率(模型利用)
    def calculate_probabilities(self, input_data):
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                # 取出每个特征值的期望和标准差
                mean, stdev = value[i]
                # 计算概率密度
                probabilities[label] *= self.gaussian_probability(input_data[i], mean, stdev)
        return probabilities

    # 预测分类
    def predict(self, X_test):
        label = sorted(self.calculate_probabilities(X_test).items(), key=lambda x: x[-1])[-1][0]
        return label
    
    # 计算正确率（模型评估，即AUC）
    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1

        return right / float(len(X_test))

In [6]:
model = NaiveBayes()

In [7]:
# 训练数据
model.fit(X_train, y_train)

'gaussianNB train done!'

In [8]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))

0.0


In [9]:
model.score(X_test, y_test)

1.0

# sklearn.naive_bayes

In [10]:
from sklearn.naive_bayes import GaussianNB

In [11]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [12]:
clf.score(X_test, y_test)

1.0

In [13]:
clf.predict([[4.4,  3.2,  1.3,  0.2]])

array([0.])